# Lab06-2 Fancy Softmay Classifier

- Cross Entropy
- One Hot
- Reshape

## softmax_cross_entropy_with_logis 함수 소개

### 기본 코드

```
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(lotits)
```

- logits혹은 score라고 함
- logits을 softmax에 입력하면 확률 데이터로 변환

### 기존 softmax 함수 직접 구현 코드

- Cross entropy cost/loss

```python
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis), axis=1))
```

### 개선코드

- cost 함수를 직접 구현에서 cost 함수를 재사용

```python
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
```

![](./images/17/softmax.jpg)

![](./images/17/softmax02.jpg)

## Aninimal Classification

### Softmax_cross_entropy_with_logits

### data

![](./images/17/animal_data.jpg)

- 16개 feature
  - hair
  - feathers
  - eggs
  - milk
  - airborne
  - aquatic
  - predator
  - toothed
  - backbone
  - breathes
  - venomous
  - fins
  - legs 
  - tail
  - domestic
  - catsize

In [16]:
import tensorflow as tf
x_data = [[0], [1], [2], [3], [4], [5], [6]]

sess = tf.Session()
Y_one_hot = tf.one_hot(x_data, 7)

In [17]:
a=sess.run(Y_one_hot)
a

array([[[ 1.,  0.,  0.,  0.,  0.,  0.,  0.]],

       [[ 0.,  1.,  0.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  1.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  1.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  1.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.,  0.,  1.]]], dtype=float32)

In [18]:
a.shape

(7, 1, 7)

In [19]:
sess.run(tf.reshape(tf.one_hot(x_data, 7), [-1, 7]))

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.]], dtype=float32)

- 입력 데이터가 rank N이면 one_hot함수는 rank N+1을 반환
- 새로운 축은 마지막 axis에 추가됨
- 입력 데이터의 shape은 (7, 1)
- 마지막 축은 (7, 1, 7)
- reshape은 공간을 만들고 배열 데이터를 재배치하는 함수임
- 7X7 공간을 만들고 데이터를 추가하는 작업
- reshape의 두번째 인자는 shape임
- shape의 -1은 전체 데이터를 의미

### 요약
- tf.one_hot으로 숫자 항을 one_hot encoding 포멧으로 변환시킴
- tf.one_hot은 마지막 축에 차원을 추가하여 예상한 포멧과 다른 데이터 출력
- tf.reshape으로 차원 조정

In [204]:
import numpy as np
import tensorflow as tf

In [205]:
xy = np.loadtxt('./data/data-04-zoo.csv', delimiter=',', dtype=np.float32 )
x = xy[:, :-1]
y = xy[:, [-1]]

In [206]:
X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1]) # 0 ~ 6

nb_classes = 7
Y_one_hot = tf.one_hot(Y, nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

W = tf.Variable(tf.random_normal([16, nb_classes]), name = 'weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

In [207]:
# tf.nn.softmax
# softmax = exp(logits) / reduce_sum(exp(lgits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)


In [208]:
prediction = tf.argmax(hypothesis, 1) # one_hot을 label로 변환함

In [209]:
correct_prediction = tf.equal(prediction, tf.arg_max(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [210]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(20001):
        sess.run(optimizer, feed_dict={X:x, Y:y})
        if step % 5000 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={X:x, Y:y})
            print("Step: {:5}\tLoss: {:.3f}\tAcc:{:.2%}".format(step, loss, acc))
            
    pred = sess.run(prediction, feed_dict={X:x})
    for p, y in zip(pred, y.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p==int(y), p, int(y)))
        


Step:     0	Loss: 5.035	Acc:14.85%
Step:  5000	Loss: 0.025	Acc:100.00%
Step: 10000	Loss: 0.013	Acc:100.00%
Step: 15000	Loss: 0.009	Acc:100.00%
Step: 20000	Loss: 0.007	Acc:100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[True] Prediction: 3 True Y: 3
[True] Prediction: 6 True Y: 6
[True] Prediction: 6 True Y: 6
[True] Prediction: 6 True Y: 6
[True] Prediction: 1 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[True] Prediction: 1 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[True] Prediction: 5 True Y: 5
[True] Prediction: 4 True Y: 4
[True] Predicti

In [211]:
import datetime
print(datetime.datetime.now())

2017-06-05 11:45:15.873257
